In [1]:
!pip install langchain tiktoken pymupdf wikipedia unstructured openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

In [14]:
import os
from datetime import date, datetime
import json
import sys

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain import PromptTemplate, LLMChain
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.document_loaders import WikipediaLoader, DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator

In [34]:
OPENAI_API_KEY = "yourownkey"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# docsLoader = WikipediaLoader(query="Tamarin", load_max_docs=2).load()
# len(docs)

2

In [71]:
wikiLoader = WikipediaLoader(query="Tamarin", load_max_docs=2)

In [81]:
!mkdir Docs # Will create folder "Docs" if not exist
docsLoader = DirectoryLoader("Docs/") # Put PDFs here

mkdir: cannot create directory ‘Docs’: File exists


In [ ]:
# Load all documents into vector storage
index = VectorstoreIndexCreator().from_loaders([docsLoader,wikiLoader])

In [82]:
def generate_template():
    """
    Custom prompt for bot instruction.
    """

    template =  """Please answer based on context below (delimited by triple backticks)
    If the question does not relate to any of the context, just say that you don't know the answer.
    Please answer in English
    ```
    {context}
    ```"""

    general_user_template = "Question:```{question}```"
    general_system_template  = template
    messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
    ]
    qa_prompt = ChatPromptTemplate.from_messages(messages)
    return qa_prompt

In [88]:
# Define object for LLM, Memory, & Retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=0, openai_api_key=OPENAI_API_KEY)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=3800, memory_key="chat_history", return_messages=True)
retriever = index.vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [84]:
# Csutom prompt template for condense_question_prompt. Just omit condense_question_prompt parameter from chain if not using custom prompt.
template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in English language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:'
condense_question_prompt = PromptTemplate(input_variables=['chat_history', 'question'],
                                          output_parser=None,
                                          partial_variables={},
                                          template=template,
                                          template_format='f-string',
                                          validate_template=True)

In [89]:
# Define chain object for Conversational Retrival Chain
chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False,
        return_generated_question=False,
        memory=memory,
        condense_question_prompt=condense_question_prompt,
        combine_docs_chain_kwargs={"prompt": generate_template()},
        verbose=True
    )

In [90]:
# Start the conversation
while True:
  query = input("Prompt: ")
  if query in ['quit', 'q', 'exit']:
    sys.exit()
  result = chain({"question": query})
  print(result['answer'])

Prompt: tell me about tamarin


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Please answer based on context below (delimited by triple backticks)
    If the question does not relate to any of the context, just say that you don't know the answer. 
    Please answer in English
    ```
    == Behavior and reproduction ==
Tamarins are inhabitants of tropical rainforests and open forest areas. They are diurnal and arboreal, and run and jump quickly through the trees. Tamarins live together in groups of up to 40 members consisting of one or more families. More frequently, though, groups are composed of just three to nine members.
Tamarins are omnivores, eating fruits and ot

== Behavior and reproduction ==
Tamarins are inhabitants of tropical rainforests and open forest areas. They are diurnal and arboreal, and run and jump quickly through the trees. Tamarins live together in groups of up to 40 members consisting of one or mo

KeyboardInterrupt: ignored